<a href="https://colab.research.google.com/github/Anakin771/Kradad_MainAssistingProgram/blob/master/GKNE_MAP_Online_v1_0_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Welcome

**This is the online edition of the GKNE - Main Assisting Program version 1.0.0!**

This Python-Notebook includes all the neccessary functions from the desktop edition of the program and is derived from the *Console Edition*.

You start by running the code cell labeled **"Start the Program!"** first, which will import important files into the running session, then, 
you can use the indexes on your right to search for the command you wish to run!

**Take note:** This is considered a *Lite* Edition, and very little to no error protection, so please be careful when giving inputs.

Thank you for using GKNE-MAP Online Edition!

In [ ]:
#@title Start the Program! (Run this first)
 
"""
***********************************************
 
Author: MontyGUI
Last Edit: 23/4/2021
 
Description
This script includes arithmetics functions of rounding a number
 
***********************************************
"""
import math
 
 
def round_half_up(n, decimals=0):
    mtp = 10 ** decimals
    return math.floor(n * mtp + 0.5) / mtp
 
 
def round_basic(n, decimals=0):
    rounded_abs = round_half_up(abs(n), decimals)
    return math.copysign(rounded_abs, n)
 
 
"""
***********************************************
 
Author: MontyGUI
 
Description
This script includes every functions that revolves
around boss's stats, such as generating a single boss stats,
generating group boss, etc.
 
***********************************************
"""
import math
import random
 
 
# Boss HP-by-LV Multiplier
EARLY_BOSS_HP_MTP = 700
MID_BOSS_HP_MTP = 850
LATE_BOSS_HP_MTP = 1000
 
# Multi-Boss-Fight Stats-Down Multiplier
MULTIPLE_BOSS_MTP = 0.8
 
# Rewards Multiplier
EARLY_XP_MULTIPLIER = 1300
MID_XP_MULTIPLIER = 1400
LATE_XP_MULTIPLIER = 1500
MONEY_MULTIPLIER = 100
 
# Difficulty names-to-rate tables...
DIFF_TO_LV_MAPPING = {
    "noob": -5,
    "easy": -3,
    "normal": 0,
    "hard": 3,
    "hardcore": 6
}
DIFF_TO_COIN_MAPPING = {
    "noob": -5,
    "easy": -1,
    "normal": 0,
    "hard": 3,
    "hardcore": 6
}
DIFF_TO_HP_MAPPING = {
    "noob": -2,
    "easy": -1,
    "normal": 0,
    "hard": 3,
    "hardcore": 8
}
DIFF_TO_ITEM_MAPPING = {
    "noob": "0",
    "easy": "1",
    "normal": "1 or 2",
    "hard": "2",
    "hardcore": "3"
}
 
 
def round_half_up(n, decimals=0):
    mtp = 10 ** decimals
    return math.floor(n * mtp + 0.5) / mtp
 
 
def round_basic(n, decimals=0):
    rounded_abs = round_half_up(abs(n), decimals)
    return math.copysign(rounded_abs, n)
 
 
def random_boss_stat(party_avg_lv, player_num, diff=None, show_stat=True):
 
    input_diff = True
 
    # Default value for diff if unspecified
    if diff is None:
        input_diff = False
        diff = "normal"
 
    # Adjust Difficulty
    hp_diff_rate, boss_lv = adjust_difficulty(party_avg_lv, diff)
    if hp_diff_rate is None or boss_lv is None:
        return
 
    # Determine Boss' HP
    boss_hp = calculate_boss_hp(boss_lv, party_avg_lv, hp_diff_rate, player_num)
 
    # Calculate Reward
    xp_reward, money_reward, item_reward = calculate_reward(boss_lv, party_avg_lv, diff)
 
    # Determine Boss Stats' Roll Point
    boss_roll_pt = boss_lv if boss_lv >= 10 else boss_lv * 5
    boss_stat_mtp = 100 if boss_lv >= 10 else 20
 
    # Create Boss Dictionary for returning
    boss = {
        "DIFFICULTY": diff,
        "LV": boss_lv,
        "HP": boss_hp,
        "PATK": 0,
        "MATK": 0,
        "PDEF": 0,
        "MDEF": 0
    }
 
    # Roll Choices
    roll_choices = ["PATK", "MATK", "PDEF", "MDEF"]
 
    # Randomize Stat Points
    stats_rand = random.choices(roll_choices, k=boss_roll_pt)
    for stat in stats_rand:
        boss[stat] += boss_stat_mtp
 
    # Show stats if set enabled
    if show_stat:
        print("-------------------------------------------------")
        print(f" BOSS - LV. {boss_lv}")
        if input_diff:
            print(f" {diff.upper()} DIFFICULTY")
        print()
        print(" STATS:")
        print(f" HP: {boss_hp}")
        print(f" P. ATK: {boss['PATK']}")
        print(f" M. ATK: {boss['MATK']}")
        print(f" P. DEF: {int(boss['PDEF'] / 2)}")
        print(f" M. DEF: {int(boss['MDEF'] / 2)}")
        print("-------------------------------------------------")
        print(" Rewards* :")
        print(
            f"  • {xp_reward} XP\n"
            f"  • {money_reward} (C)\n"
            f"  • {item_reward} Item(s)\n"
        )
        print(" * Rewards on a scenario case that no one died,\n"
              "and no +20% Bonus from declining dropped item.")
        print("-------------------------------------------------")
 
    return boss
 
 
def random_boss_stat_multi(party_avg_lv, player_num, qty, diff=None, show_stat=True):
 
    input_diff = True
 
    # Default value for diff if unspecified
    if diff is None:
        input_diff = False
        diff = "normal"
 
    hp_diff_rate, boss_lv = adjust_difficulty(party_avg_lv, diff)
 
    # Calculate Reward
    xp_reward, money_reward, item_reward = calculate_reward(boss_lv, party_avg_lv, diff)
 
    # Determine Bosses' HP
    boss_hp = int(calculate_boss_hp(boss_lv, party_avg_lv, hp_diff_rate, player_num) / 3)
 
    # Initial Variable Setup
    boss_list = []
    count = 0
    reduce_mtp = 0.8
 
    while count < qty:
        single_boss_stat = random_boss_stat(party_avg_lv, player_num, diff=diff, show_stat=False)
        current_boss = {
            "LV": single_boss_stat["LV"],
            "DIFFICULTY": diff,
            "HP": boss_hp,
            "PATK": int(round_basic(single_boss_stat["PATK"] * reduce_mtp, -1)),
            "MATK": int(round_basic(single_boss_stat["MATK"] * reduce_mtp, -1)),
            "PDEF": int(round_basic(single_boss_stat["PDEF"] * reduce_mtp, -1)),
            "MDEF": int(round_basic(single_boss_stat["MDEF"] * reduce_mtp, -1))
        }
 
        boss_list.append(current_boss)
        count += 1
 
    if show_stat:
        boss_count = 1
 
        print("**********************************************")
        print("     -------- MULTI-BOSS FIGHT --------    ")
        if input_diff:
            print(f" {diff.upper()} DIFFICULTY")
        print()
        for boss in boss_list:
            print(f" Boss #{boss_count} Stats:")
            for stat, value in boss.items():
                # Skip Difficulty (This is added for compatibility with Graphical Edition)
                if stat == "DIFFICULTY":
                    continue
 
                if stat == "LV":
                    print(f" LV. {value}")
                elif stat == "PDEF" or stat == "MDEF":
                    print(f" {stat}: {int(value / 2)}")
                else:
                    print(f" {stat}: {value}")
 
            print()
            boss_count += 1
        print("-------------------------------------------------")
        print(" Rewards* :")
        print(
            f"  • {xp_reward} XP\n"
            f"  • {money_reward} (C)\n"
            f"  • {item_reward} Item(s)\n"
        )
        print(" * Rewards on a scenario case that no one died,\n"
              "and no +20% Bonus from declining dropped item.\n")
        print("**********************************************")
 
    return boss_list
 
 
def calculate_boss_hp(boss_lv, pty_avg, hp_diff_rate, player_num):
    if boss_lv <= 30:
        boss_hp = (pty_avg + hp_diff_rate) * player_num * EARLY_BOSS_HP_MTP
    elif boss_lv <= 60:
        boss_hp = (pty_avg + hp_diff_rate) * player_num * MID_BOSS_HP_MTP
    else:
        boss_hp = (pty_avg + hp_diff_rate) * player_num * LATE_BOSS_HP_MTP
 
    return int(boss_hp)
 
 
def calculate_xp(boss_lv):
    if boss_lv <= 30:
        xp_reward = boss_lv * EARLY_XP_MULTIPLIER
    elif boss_lv <= 60:
        xp_reward = boss_lv * MID_XP_MULTIPLIER
    else:
        xp_reward = boss_lv * LATE_XP_MULTIPLIER
 
    return int(xp_reward)
 
 
def adjust_difficulty(prty_avg, diff, show_text="True"):
    # Get Boss LV Difficulty Rate
    lv_diff_rate = DIFF_TO_LV_MAPPING.get(diff, 0)
 
    # Get Boss HP Difficulty Rate
    hp_diff_rate = DIFF_TO_HP_MAPPING.get(diff, 0)
 
    # Find Bosses' Real LV
    boss_lv = prty_avg + lv_diff_rate
 
    # If calculated LV is 0 or less, flag an error an find the floor difficulty for that party.
    if boss_lv <= 0:
        least_diff = ""
        for diff_name, lv in DIFF_TO_LV_MAPPING.items():
            if prty_avg + lv > 0:
                least_diff = diff_name
                break
 
        if show_text:
            print(" Error: Too low difficulty!")
            print(f" For LV {prty_avg} party,"
                  f" Boss Difficulty must be at least on {least_diff.capitalize()}.")
        return None, None
 
    return hp_diff_rate, boss_lv
 
 
def calculate_reward(boss_lv, prty_avg, diff):
    # Calculate XP Reward
    xp_reward = calculate_xp(boss_lv)
 
    # Get Boss Coin Reward Difficulty Rate
    money_diff_rate = DIFF_TO_COIN_MAPPING.get(diff, 0)
 
    # Calculate Money Reward
    money_reward = (prty_avg + money_diff_rate) * MONEY_MULTIPLIER
 
    # Get Boss Item Reward Difficulty Rate
    item_reward = DIFF_TO_ITEM_MAPPING.get(diff, 0)
 
    return xp_reward, money_reward, item_reward
 
"""
***********************************************
 
Author: MontyGUI
 
Description
This script includes every functions that revolves around
generating items, including stats randomization, item crafting, etc.
 
***********************************************
"""
 
import random
import math
 
QUALITY_MAPPING = {}
QUALITY_MAPPING.update(dict.fromkeys([1, 2, 3, 18, 19, 20], -7))  # Bad Quality
QUALITY_MAPPING.update(dict.fromkeys(range(6, 16), 0))  # Normal Quality
QUALITY_MAPPING.update(dict.fromkeys([4, 5, 16, 17], 7))  # Great Quality
 
QUALITY_TEXT_MAPPING = {
    -7: "BAD",
    0: "NORMAL",
    7: "GREAT"
}
 
MATERIAL_TYPE_MAPPING = {
    "wpn": 1,
    "amr": 10,
    "acc": 100
}
 
CRAFT_TYPE_MAPPING = {
    3: "wpn",
    30: "amr",
    300: "acc"
}
 
ITEM_TYPE_MAPPING = {
    "wpn": "Weapon",
    "amr": "Armor",
    "acc": "Accessories"
}
 
WPN_WEIGHT = [12, 32, 32, 12, 12]  # Weapon Random Weight
AMR_WEIGHT = [26, 11, 11, 26, 26]  # Armor Random Weight
 
DEVIATION = 12
 
 
def random_item_stat(item_lv, item_type, quality_rate=None, show_stat=True):
    """
    Random the stats for an item
    :param item_lv:
    :param item_type:
    :param quality_rate:
    :param show_stat:
    :return:
    """
 
    if quality_rate is None:
        quality_rate = 0
        quality_text = ""
    else:
        if QUALITY_TEXT_MAPPING.get(quality_rate, None) is None:
            print("Sorry, you do not input the right quality rate!")
            return None
        quality_text = f"{QUALITY_TEXT_MAPPING.get(quality_rate, None)} QUALITY "
 
    # Set the number of Stat Roll: Item's LV * 5 for LV 1-9, and Item's LV for LV 10+
    # Also, if the item has quality rate, it is also added to the roll point here
    item_stat_pt = (item_lv + quality_rate) if item_lv >= 10 else (item_lv + quality_rate) * 5
 
    # Set the number of Stat Point Multiplier: 10 for LV 1-9, and 50 for LV 10+
    item_stat_mtp = 50 if item_lv >= 10 else 10
 
    roll_choice = ["HP", "PATK", "MATK", "PDEF", "MDEF"]
 
    if item_type == "wpn":
        rand_variant = round(random.gauss(0, DEVIATION))
        variant = int(min(abs(rand_variant), 32))
        variant = int(math.copysign(variant, rand_variant))
        roll_weight = WPN_WEIGHT
        roll_weight[1] += variant
        roll_weight[2] -= variant
    elif item_type == "amr":
        roll_weight = AMR_WEIGHT
    else:
        roll_weight = [20, 20, 20, 20, 20]
 
    # Build Item's Stat Points dictionary
    item = {
        "LV": item_lv,
    }
 
    if item_type is not None:
        item.update({"TYPE": item_type})
 
    if quality_rate is not None:
        item.update({"QUALITY": quality_rate})
 
    # This line is SOOOO REDUNDANT! But is the author's weird preferences ordering dictionary lol
    item.update({
        "HP": 0,
        "PATK": 0,
        "MATK": 0,
        "PDEF": 0,
        "MDEF": 0
    })
 
    # Random the category for the current stat point
    item_rand = random.choices(roll_choice, weights=roll_weight, k=item_stat_pt)
    for stat in item_rand:
        item[stat] += 1 * item_stat_mtp
 
    # Display the stat (if needed)
    if show_stat:
        item_type_text = f"{ITEM_TYPE_MAPPING.get(item_type).upper()} - " if item_type is not None else ""
 
        print("---------------------------------")
        print(f" ITEM STAT POINTS\n({quality_text}{item_type_text}LV. {item_lv})")
        print(f" HP: +{item['HP']} pts.")
        print(f" P. ATK: +{item['PATK']} pts.")
        print(f" M. ATK: +{item['MATK']} pts.")
        print(f" P. DEF: +{item['PDEF']} pts.")
        print(f" M. DEF: +{item['MDEF']} pts.")
        print("---------------------------------")
 
    """
    # Return Format (For usage in other parts)
    item = {
        "LV": lv,
        "TYPE": type,
        "QUALITY": quality,
        "HP": hp,
        "PATK": patk,
        "MATK": matk,
        "PDEF": pdef,
        "MDEF": mdef
    }
    """
    return item
 
 
def craft_item(material_a, material_b, material_c, player_lv, show_stat=True):
    """
    Craft an item based on given material type
    :param material_a:
    :param material_b:
    :param material_c:
    :param player_lv:
    :param show_stat:
    :return:
    """
    type_sum =\
        MATERIAL_TYPE_MAPPING.get(material_a, -1000) \
        + MATERIAL_TYPE_MAPPING.get(material_b, -1000) \
        + MATERIAL_TYPE_MAPPING.get(material_c, -1000)
 
    if type_sum < 0:
        print("Sorry, you didn't input all material types correctly.")
        return None
 
    crafted_type = CRAFT_TYPE_MAPPING.get(type_sum, random.choice(["wpn", "amr", "acc"]))
    crafted_quality = QUALITY_MAPPING.get(random.randint(1, 20))
 
    crafted_item = random_item_stat(
        player_lv,
        quality_rate=crafted_quality,
        item_type=crafted_type,
        show_stat=show_stat
    )
 
    return crafted_item
 
 
"""
***********************************************
 
Author: MontyGUI
 
Description
This script includes every functions that revolves
around player's stats, including initial stats generation,
calculate levels, etc.
 
***********************************************
"""
 
import random
from os import system, name
 
 
def clear():
    if name == "nt":
        _ = system('cls')
    else:
        _ = system('clear')
 
 
STARTING_BASE_ROLL = 40
STARTING_MTP = 5
 
BONUS_BASE_ROLL = 20
BONUS_MTP = 5
 
EARLY_REQ_XP_RATE = 1.5
MID_REQ_XP_RATE = 1.65
LATE_REQ_XP_RATE = 1.8
 
ITEM_DECLINED_BONUS_MTP = 1.2
 
FALLEN_ALLY_PENALTY = 0.1
 
 
def random_char_stat_single(base_roll_point, multiplier, header_text=None, show_stat=True):
    """
    Randomize a character stat points only once upon given factors.
    :param base_roll_point: The number of times to random a category of stats
    :param multiplier: The number which is multiplied after one roll point is assigned to a category
    :param header_text: Optional - The text that will be written in the console as a header.
    :param show_stat: Optional - Triggers displaying the randomized stat points if set to True
    :return: Returns a dictionary of a randomized character stat points
    """
    if header_text is not None:
        print(header_text)
 
    count = 0
 
    hp = 0
    p_atk = 0
    m_atk = 0
    p_def = 0
    m_def = 0
    free_stat = 0
 
    while count < base_roll_point:
        stat_cat = random.randint(1, 6)
 
        if stat_cat == 1:
            hp += 1 * multiplier
        elif stat_cat == 2:
            p_atk += 1 * multiplier
        elif stat_cat == 3:
            m_atk += 1 * multiplier
        elif stat_cat == 4:
            p_def += 1 * multiplier
        elif stat_cat == 5:
            m_def += 1 * multiplier
        elif stat_cat == 6:
            free_stat += 1 * multiplier
 
        count += 1
 
    if show_stat:
        print("-----------------------------")
        print(f" HP: {hp} pts.")
        print(f" P. ATK: {p_atk} pts.")
        print(f" M. ATK: {m_atk} pts.")
        print(f" P. DEF: {p_def} pts.")
        print(f" M. DEF: {m_def} pts.")
        print(f" FREE: {free_stat} pts.")
        print("-----------------------------")
 
    return {
        'HP': hp,
        'PATK': p_atk,
        'MATK': m_atk,
        'PDEF': p_def,
        'MDEF': m_def,
        'FREE': free_stat
    }
 
 
def random_char_stat_full(base_roll_point, multiplier, header_text=""):
    """
    Randomize a character stat points upon given factors. (CONSOLE EDITION)
    :param base_roll_point: The number of times to random a category of stats
    :param multiplier: The number which is multiplied after one roll point is assigned to a category
    :param header_text: Optional - The text that will be written in the console as a header.
    :return: Returns a dictionary of a randomized character stat points that has been chosen by the user;
    None if the user cancels.
    """
    while True:
 
        sample_stat_points = random_char_stat_single(
            base_roll_point,
            multiplier,
            header_text=header_text,
            show_stat=True
        )
 
        try:
            confirm = str(input("Choose these stats? (Y/N/Cancel): ")).lower()
            if confirm == 'y':
                return sample_stat_points
            elif confirm == 'cancel':
                print("Action Canceled")
                return None
            else:
                clear()
                continue
        except TypeError:
            print("An error has occurred, try again.")
 
 
def random_starting_char():
    """
    Begin randomizing new character's Stat Points
    :return: Dictionary of Starting Stats and Bonus Rate;
    None if the action is canceled.
    """
    starting = random_char_stat_full(
        STARTING_BASE_ROLL,
        STARTING_MTP,
        header_text="------------ Step 1: Randomizing Starting Stats ------------"
    )
    if starting is None:
        return None
 
    bonus = random_char_stat_full(
        BONUS_BASE_ROLL,
        BONUS_MTP,
        header_text="------------ Step 2: Randomizing Bonus Rate ------------"
    )
    if bonus is None:
        return None
 
    clear()
    print("***********************\n"
          "       YOUR STATS:     \n"
          "***********************\n")
 
    print(" Starting Stat Points:")
    for stat, pts in starting.items():
        print(f" {stat}: {pts} pts.")
    print("----------------------")
    print(" Bonus Rate Points:")
    for stat, pts in bonus.items():
        print(f" {stat}: {pts} pts.")
 
    return {
        "starting": starting,
        "bonus": bonus
    }
 
 
def calculate_level_up(char_lv, current_xp, gained_xp, fallen=0, item_accepted=True, show_stat=True):
    new_char_lv = char_lv
    req_xp = cal_req_xp(new_char_lv)
 
    # Reduce gained XP due to fallen party member
    fallen_penalty_rate = 1 - (fallen * FALLEN_ALLY_PENALTY)
    gained_xp = max(gained_xp * fallen_penalty_rate, 0)
 
    # +20% XP bonus for players who declined dropped item
    gained_xp *= ITEM_DECLINED_BONUS_MTP if not item_accepted else 1
 
    # For LV 10 and lower, XP Gained is x2
    gained_xp *= 2 if char_lv <= 10 else 1
 
    # Calculate Level Up
    total_xp = current_xp + int(round_basic(gained_xp))
    gained_sp = 0
    while total_xp >= req_xp:
        new_char_lv += 1
        total_xp -= req_xp
        req_xp = cal_req_xp(new_char_lv)
 
        if new_char_lv % 10 == 0:
            gained_sp += 2
        elif new_char_lv % 5 == 0:
            gained_sp += 1
 
    notice = ""
    if new_char_lv <= char_lv:
        if gained_xp <= 0 < fallen:
            notice = "Too many players have fallen."
        else:
            notice = "Your total XP does not reach target XP."
 
    if show_stat:
        extra_cal = False
 
        if char_lv <= 10:
            print(" (Player with LV 10 or lower gets x2 XP)")
            extra_cal = True
 
        if fallen > 0:
            print(f" ({fallen} Party member(s) have fallen, -{min(int(fallen * 10), 100)}% XP Penalty)")
            extra_cal = True
 
        if not item_accepted:
            print(f" (Item Declined: +{int(round_basic((ITEM_DECLINED_BONUS_MTP - 1) * 100))}% XP)")
            extra_cal = True
 
        if gained_xp <= 0 < fallen:
            print(
                "--------------------------------\n"
                " Too many players have fallen!\n"
                " You do not gain any XP...\n"
                "--------------------------------\n"
            )
        elif new_char_lv > char_lv:
            print("---------------------------------")
            print(" You have leveled up!")
            print(f" Your Character's LV: {char_lv} -> {new_char_lv}")
            print(f" Remainder XP: {current_xp} -> {total_xp}")
            print(f" To next LV: {req_xp} XP")
            if gained_sp > 0:
                print(f" You have received {gained_sp} Skill Point(s).")
            print("---------------------------------")
        else:
            print("---------------------------------")
            print(" Sorry, but you did not level up...")
            print(f" Your Current XP: {current_xp} -> {total_xp}")
            print("---------------------------------")
 
        if extra_cal:
            print(" * Bonus Calculation goes from top to bottom.")
 
    return {
        "lv_up": bool(new_char_lv > char_lv),
        "item_decl": bool(not item_accepted),
        "fallen": fallen,
        "fallen_penalty": min(int(fallen * 10), 100),
        "old": {
            "lv": char_lv,
            "rem_xp": current_xp
        },
        "new": {
            "lv": new_char_lv,
            "rem_xp": total_xp
        },
        "gained_xp": gained_xp,
        "gained_sp": gained_sp,
        "note": notice
    }
 
 
def cal_req_xp(char_lv):
    if char_lv <= 30:
        return int(round_basic(100 * (char_lv ** EARLY_REQ_XP_RATE), -2))
    elif char_lv <= 60:
        return int(round_basic(100 * (char_lv ** MID_REQ_XP_RATE), -2))
    else:
        return int(round_basic(100 * (char_lv ** LATE_REQ_XP_RATE), -2))

print("Initiation Complete! You can now use the programs below!")

# Player: New Character

The code cell below will generate a new character stats. It doen't take any initial arguments, but you must make decisions in the console printed below:

In [ ]:
#@title New Character

new_char = random_starting_char()

# Player: Calculate Level

The code cell below calculate the character's Level progress. It takes the following arguments:

* Your Character's Current Level
* Current Remainder XP
* XP Gained
* No. of Fallen Ally
* Item Acception Decision

Please not that since this is only the lite version for online platform, error protection is absent, and it can occur if you input invalid value(s)

In [ ]:
#@title Calculate Level

current_lv =  1 #@param {type:"number"}
remainder_xp = 200 #@param {type:"number"}
xp_gained = 12500 #@param {type:"number"}
fallen_ally = 1 #@param {type:"number"}
item_decision = 'Declined' #@param ["Accepted", "Declined", "No Item"]

if item_decision in ["Accepted"]:
  item_acc = True
else:
  item_acc = False

result = calculate_level_up(current_lv, remainder_xp, xp_gained, fallen_ally, item_acc)

# Boss: Generate Boss Fight

The code cell below generates a boss fight of any type (Single or Multi-Boss). It takes the following arguments:

* Party's Average Level
* Party Size
* Number of Bosses
* Difficulty

As stated before, there are no error protection, so please keep your inputs as sensible as possible to avoid errors

In [ ]:
#@title Generate Boss Fight

party_avg_lv = 50 #@param {type:"number"}
party_size = 5 #@param {type:"number"}
boss_amount = 1 #@param {type:"number"}
difficulty = "Normal" #@param ["Noob", "Easy", "Normal", "Hard", "Hardcore"]

difficulty.lower()

if boss_amount > 1:
  boss = random_boss_stat_multi(party_avg_lv, party_size, boss_amount, difficulty)
else:
  boss = random_boss_stat(party_avg_lv, party_size, difficulty)

# Item: Generate Dropped Item
The code cell below generates a dropped item from boss fight. It takes the following arguments:

* Boss LV
* Item Type

As stated before, there are no error protection, so please keep your inputs as sensible as possible to avoid errors

In [ ]:
#@title Generate Dropped Item

boss_lv = 15 #@param {type:"number"}
item_type = "Weapon" #@param ["Weapon", "Armor", "Accessories"]

item_type_dict = {
    "Weapon": "wpn",
    "Armor": "amr",
    "Accessories": "acc"
}

translated_type = item_type_dict.get(item_type, None) 

item = random_item_stat(boss_lv, translated_type)

# Item: Craft Item

The code cell below generates a crafted item. It takes the following arguments:

* Player's LV
* 3 Crafting Material Type (Weapon/Armor/Accessories)

There are internal error protection this time, but please keep your inputs as sensible as possible to avoid errors.

In [ ]:
#@title Craft Item

player_lv = 15 #@param {type:"number"}
material_1 = "" #@param ["", "Weapon", "Armor", "Accessories"]
material_2 = "" #@param ["", "Weapon", "Armor", "Accessories"]
material_3 = "" #@param ["", "Weapon", "Armor", "Accessories"]

item_type_dict = {
    "Weapon": "wpn",
    "Armor": "amr",
    "Accessories": "acc"
}

translated_mat1 = item_type_dict.get(material_1, None)
translated_mat2 = item_type_dict.get(material_2, None)
translated_mat3 = item_type_dict.get(material_3, None)

item = craft_item(translated_mat1, translated_mat2, translated_mat3, player_lv)